In [9]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

driver = webdriver.Chrome() 

driver.implicitly_wait(10)
driver.get('https://www.ffiec.gov/census/Default.aspx')
driver.find_element_by_xpath('//*[@id="MsaImgButton"]').click()

In [ ]:
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import sqlite3
con = sqlite3.connect("./FFIECTable.db")

table_dict = dict()

MSAlen = len(driver.find_elements_by_xpath('//*[@id="MSAListBox"]/option'))

for i in range(266, MSAlen):
    print 'MSA {0}'.format(i + 1)
    driver.find_element_by_xpath('//*[@id="MSAListBox"]/option[{0}]'.format(i + 1)).click()
    driver.find_element_by_xpath('//*[@id="MSACensusDemographicImgButton"]').click()
    back = 1
    for text in ['Demographic', 'Income', 'Population', 'Housing']:
        print '\tDownloading table:', text
        page = 1
        if not text == 'Demographic':
            driver.find_element_by_link_text(text).click()
            back += 1
        while True:
            print '\t\tPage number {0}'.format(page)
            table = driver.find_element_by_xpath('//*[@id="aspnetForm"]')
            df = pd.read_html(table.get_attribute('innerHTML'), header=0)[0]
            df.to_sql(text, con, if_exists='append')
            try:
                table_dict[text] = table_dict[text].append(df, ignore_index=True)
            except KeyError, TypeError:
                table_dict[text] = df
            try:
                driver.find_element_by_link_text('{0}'.format(page + 1)).click()
                page += 1
                back += 1
            except NoSuchElementException:
                break
            time.sleep(1)
            
    driver.execute_script('window.history.go(-{0})'.format(back))

In [6]:
import sqlite3
import pandas as pd

con = sqlite3.connect("./FFIECTable.db")

In [ ]:
housing = pd.read_sql_query('SELECT * FROM Housing', con)
population = pd.read_sql_query('SELECT * FROM Population', con)
demographic = pd.read_sql_query('SELECT * FROM Demographic', con)
income = pd.read_sql_query('SELECT * FROM Income', con)

In [ ]:
housing.to_csv('./housing.csv', index=False)
population.to_csv('./population.csv', index=False)
demographic.to_csv('./demographic.csv', index=False)
income.to_csv('./income.csv', index=False)

In [ ]:
housing.to_pickle('./housing.pkl')
population.to_pickle('./population.pkl')
demographic.to_pickle('./demographic.pkl')
income.to_pickle('./income.pkl')

In [ ]:
con.close()